In [1]:
import numpy as np, pandas as pd
import seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import pickle
import tensorflow
from tensorflow import keras # type: ignore
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Ingest Data

In [17]:
churn_data=pd.read_csv("Churn_Modelling.csv")
df = churn_data.copy() 

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Data info and null values

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Remove columns having unique values

In [18]:
df.drop(columns=df.iloc[:,:3].columns,axis=1,inplace=True)

### Import module for pre-processing

In [19]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
label_encode = LabelEncoder()
onehot_encode = OneHotEncoder(sparse_output=False).set_output(transform='pandas')
scaling = MinMaxScaler()

### Labeling and Scaling Data 

In [20]:
df['Gender'] = label_encode.fit_transform(df['Gender'])
onehot_feature = onehot_encode.fit_transform(X=df[['Geography']]) # type: ignore
df = pd.concat([df.drop(columns='Geography'),onehot_feature],axis=1)

In [21]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


### Splitting the data

In [22]:
X = df.drop(columns='EstimatedSalary',axis=1)
y = df['EstimatedSalary']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=12)

In [23]:
X_train.iloc[:,[0,2,3,4]] = scaling.fit_transform(X_train.iloc[:,[0,2,3,4]])
X_test.iloc[:,[0,2,3,4]] = scaling.transform(X_test.iloc[:,[0,2,3,4]])

In [41]:
with open('label_encode_reg.pkl','wb') as file:
    pickle.dump(label_encode,file)
with open('onehot_encode_reg.pkl','wb') as file:
    pickle.dump(onehot_encode,file)
with open('scaling_reg.pkl','wb') as file:
    pickle.dump(scaling,file)

In [34]:
from tensorflow.keras import Sequential

model = Sequential([
    keras.layers.Dense(128,activation='relu',input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(1)
])

In [35]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,033 (47.00 KB)

 Trainable params: 12,033 (47.00 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001)

In [37]:
model.compile(optimizer=opt,loss='mean_absolute_error',metrics=['mae'])

In [38]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [40]:
history = model.fit(
    x=X_train,y=y_train,
    validation_data = (X_test,y_test),
    epochs=100,
    callbacks=early_stop
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - loss: 50445.4023 - mae: 50445.4023 - val_loss: 49666.4922 - val_mae: 49666.4922
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - loss: 50206.4180 - mae: 50206.4180 - val_loss: 49638.8984 - val_mae: 49638.8984
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - loss: 51001.3984 - mae: 51001.3984 - val_loss: 49691.5195 - val_mae: 49691.5195
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - loss: 49953.4688 - mae: 49953.4688 - val_loss: 49679.9414 - val_mae: 49679.9414
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - loss: 50588.8125 - mae: 50588.8125 - val_loss: 49684.8398 - val_mae: 49684.8398
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step - loss: 50404.2383 - mae: 50404.2383 - val_loss: 49655.1680 - val_mae: 49655.1680
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - loss: 49981.9453 - mae: 49981.9453 - val_loss: 49631.1523 - val_mae: 49631.1523
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 

In [42]:
model.save('model_reg.keras')

In [43]:
from tensorflow.keras.models import load_model
model = load_model('model_reg.keras')

In [45]:
X_train.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [61]:
# input_data = pd.DataFrame([(800,'Male',30,3,50000,2,1,0,1)],columns=X_train.iloc[:,:9].columns)

In [98]:
input_data = {
    'CreditScore':500,
    'Gender':'Female',
    'Age':24,
    'Tenure':3,
    'Balance':23000,
    'NumOfProducts':1,
    'HasCrCard':1,
    'IsActiveMember':1,
    'Exited':0,
    'Geography':'Germany' 
}

In [99]:
input_data = pd.DataFrame([input_data])
input_data['Gender'] = label_encode.transform([input_data['Gender']])
geo_encode = onehot_encode.transform(input_data[['Geography']])
input_data = pd.concat([input_data.drop(columns='Geography',axis=1),geo_encode],axis=1)
input_data[['CreditScore','Age','Tenure','Balance']] = scaling.transform(input_data[['CreditScore','Age','Tenure','Balance']])
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,0.3,0,0.081081,0.3,0.096482,1,1,1,0,0.0,1.0,0.0


In [105]:
prediction = model.predict(input_data)
print(round(prediction[0][0]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
97764


In [102]:
round(22.563,2)

22.56